In [ ]:
"""
from google.colab import drive                  
drive.mount('/content/drive') 

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
1
2
3
4
5
6
7
8
9
10
11
12
13

!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive
1
2
3

import os

os.chdir("drive") 
1
2
3
4
5

!ls
"""

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
import os
import matplotlib.pyplot as plt
import numpy as np
import json
import pandas as pd
import math
from PIL import Image

In [ ]:
# import pretrained models
alexnet = torchvision.models.alexnet(pretrained = True)
resnet = torchvision.models.resnet50(pretrained = True)
vgg = torchvision.models.vgg16(pretrained = True)

# modify the last layer to 9 classes
in_features = alexnet.classifier[-1].in_features
alexnet.classifier[-1] = nn.Linear(in_features, 9)
# alexnet.eval()

in_features = resnet.fc.in_features
resnet.fc = nn.Linear(in_features, 9)
# resnet.eval()

in_features = vgg.classifier[-1].in_features
vgg.classifier[-1] = nn.Linear(in_features, 9)
# vgg.eval()

In [ ]:
# import data

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

transform_ = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

train_dir = 'train'
data_train =  datasets.ImageFolder(root = train_dir, transform = transform_,
                                       target_transform = None)
val_dir = 'val'
data_val =  datasets.ImageFolder(root = val_dir, transform = transform_,
                                       target_transform = None)

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(
  data_train,
  batch_size = 100,
  num_workers = 2,
  shuffle = True,
  pin_memory = torch.cuda.is_available()
)

val_loader = DataLoader(
  data_val,
  batch_size = 100,
  num_workers = 2,
  shuffle = True,
  pin_memory = torch.cuda.is_available()
)

In [ ]:
# use GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# train model
nb_epochs = 10
model, criterion = alexnet, nn.CrossEntropyLoss()
model.to(device)
criterion.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-2)

print("training start")
for e in range(nb_epochs):
  # Freeze the parameters except the last layer
    for p in model.features.parameters():
        p.requires_grad = False
    for p in model.classifier[0:-1].parameters():
        p.requires_grad = False

    acc_loss = 0.0
    print('epoch ' + str(e))
    for input, target in iter(train_loader):
        input, target = input.to(device), target.to(device)
        output = model(input)
        loss = criterion(output, target)
        acc_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('batch loss', loss.item())
    torch.save(model, '\model.pkl')
    print('epoch loss', acc_loss)

In [ ]:
# test model accuracy on validation set

model.eval()

predictions = torch.tensor([]).to(device)
targets = torch.tensor([]).to(device)

for input, target in iter(train_loader):
    input, target = input.to(device), target.to(device)
    pred = model(input).max(dim=1)[1]
    acc = (pred == target).float().mean().item()
    print("accuracy: " + str(acc))
    predictions = torch.cat((predictions, pred), 0)
    targets = torch.cat((targets, target), 0)